In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#madhav parashar, 500084206
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from string import punctuation
nltk.download('punkt')
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Reading the dataset
df = pd.read_csv('/content/arxiv_data.csv')

In [ ]:
df = df[:100]

In [ ]:
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [ ]:
df['text'] = df['titles'] + ' ' + df['summaries']

In [ ]:
df['text'].str.lower()

0     survey on semantic stereo matching / semantic ...
1     future-ai: guiding principles and consensus re...
2     enforcing mutual consistency of hard regions f...
3     parameter decoupling strategy for semi-supervi...
4     background-foreground segmentation for interio...
                            ...                        
95    segmentation with multiple acceptable annotati...
96    information-theoretic segmentation by inpainti...
97    are conditional gans explicitly conditional? t...
98    k-net: towards unified image segmentation sema...
99    poisoning the search space in neural architect...
Name: text, Length: 100, dtype: object

In [ ]:
#removing unwanted symbols and text
import re
def remove_unwanted(text):
  tweet=re.sub('@[^\s]+','',text)
  tweet=re.sub('http[^\s]+','',text)
  return text
df['text']=df['text'].apply(remove_unwanted)

In [ ]:
#tokenization
def token(text):
  token=re.split('W+',text)
  return token

df['text_token']=df['text'].apply(lambda x:word_tokenize(x))

In [ ]:
#stopword
stopwords=nltk.corpus.stopwords.words("english")
def remove_stopwords(text):
  out=[i for i in text if i not in stopwords]
  return out

df['text_stopwords']=df['text_token'].apply(lambda x:remove_stopwords(x))

In [ ]:
#stemming
from nltk.stem.porter import PorterStemmer
porter_stemmer=PorterStemmer()
def stemming(text):
  stem_text=[porter_stemmer.stem(word) for word in text]
  return stem_text

df['stemmed_text']=df['text_stopwords'].apply(lambda x:stemming(x))

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
word_net_leme=WordNetLemmatizer()
def lemme(text):
  lemme_text=[word_net_leme.lemmatize(word) for word in text]
  return lemme_text

df['lemmetization']=df['stemmed_text'].apply(lambda x:lemme(x))
df

,titles,summaries,terms,text,text_token,text_stopwords,stemmed_text,lemmetization
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']",Survey on Semantic Stereo Matching / Semantic ...,"[Survey, on, Semantic, Stereo, Matching, /, Se...","[Survey, Semantic, Stereo, Matching, /, Semant...","[survey, semant, stereo, match, /, semant, dep...","[survey, semant, stereo, match, /, semant, dep..."
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']",FUTURE-AI: Guiding Principles and Consensus Re...,"[FUTURE-AI, :, Guiding, Principles, and, Conse...","[FUTURE-AI, :, Guiding, Principles, Consensus,...","[future-ai, :, guid, principl, consensu, recom...","[future-ai, :, guid, principl, consensu, recom..."
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']",Enforcing Mutual Consistency of Hard Regions f...,"[Enforcing, Mutual, Consistency, of, Hard, Reg...","[Enforcing, Mutual, Consistency, Hard, Regions...","[enforc, mutual, consist, hard, region, semi-s...","[enforc, mutual, consist, hard, region, semi-s..."
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV'],Parameter Decoupling Strategy for Semi-supervi...,"[Parameter, Decoupling, Strategy, for, Semi-su...","[Parameter, Decoupling, Strategy, Semi-supervi...","[paramet, decoupl, strategi, semi-supervis, 3d...","[paramet, decoupl, strategi, semi-supervis, 3d..."
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']",Background-Foreground Segmentation for Interio...,"[Background-Foreground, Segmentation, for, Int...","[Background-Foreground, Segmentation, Interior...","[background-foreground, segment, interior, sen...","[background-foreground, segment, interior, sen..."
...,...,...,...,...,...,...,...,...
95,Segmentation with Multiple Acceptable Annotati...,Most existing deep learning-based frameworks f...,['cs.CV'],Segmentation with Multiple Acceptable Annotati...,"[Segmentation, with, Multiple, Acceptable, Ann...","[Segmentation, Multiple, Acceptable, Annotatio...","[segment, multipl, accept, annot, :, a, case, ...","[segment, multipl, accept, annot, :, a, case, ..."
96,Information-Theoretic Segmentation by Inpainti...,We study image segmentation from an informatio...,['cs.CV'],Information-Theoretic Segmentation by Inpainti...,"[Information-Theoretic, Segmentation, by, Inpa...","[Information-Theoretic, Segmentation, Inpainti...","[information-theoret, segment, inpaint, error,...","[information-theoret, segment, inpaint, error,..."
97,Are conditional GANs explicitly conditional?,This paper proposes two important contribution...,"['cs.CV', 'cs.AI']",Are conditional GANs explicitly conditional? T...,"[Are, conditional, GANs, explicitly, condition...","[Are, conditional, GANs, explicitly, condition...","[are, condit, gan, explicitli, condit, ?, thi,...","[are, condit, gan, explicitli, condit, ?, thi,..."
98,K-Net: Towards Unified Image Segmentation,"Semantic, instance, and panoptic segmentations...","['cs.CV', 'cs.AI']",K-Net: Towards Unified Image Segmentation Sema...,"[K-Net, :, Towards, Unified, Image, Segmentati...","[K-Net, :, Towards, Unified, Image, Segmentati...","[k-net, :, toward, unifi, imag, segment, seman...","[k-net, :, toward, unifi, imag, segment, seman..."


In [ ]:
# Word2Vec Embeddings
w2v_model = Word2Vec(df['text'].apply(lambda x: x.split()), window=5, min_count=1)

In [ ]:
w2v = dict(zip(w2v_model.wv.index_to_key, w2v_model.wv.vectors))

In [ ]:
# BERT Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
bert_model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def get_bert_embeddings(text):
    tokenized = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**tokenized)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        return sentence_embedding.numpy()

In [ ]:
df['bert_embeddings'] = df['text'].apply(lambda x: get_bert_embeddings(x))

In [ ]:
# Model Building
X = np.array(df['bert_embeddings'].tolist())
y = np.array(df['terms'].apply(lambda x: x.split()[0]).tolist())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(y_pred)

["['cs.CV']" "['cs.CV']" "['cs.CV'," "['cs.CV']" "['cs.CV']" "['cs.CV']"
 "['cs.CV'," "['cs.CV']" "['cs.CV']" "['cs.CV']" "['cs.CV']" "['cs.CV']"
 "['cs.CV'," "['cs.CV'," "['cs.CV'," "['cs.CV'," "['cs.CV']" "['cs.CV']"
 "['cs.CV'," "['cs.CV']"]
